In [16]:
push!(LOAD_PATH,".")
using ISCLib

In [2]:
@everywhere using DistributedArrays, NIfTI
PATH = "/run/media/ashedko/5cdd1287-7dba-4138-8a1b-91148f9f8ab5/ashedko/UNC/"
PATH_Fmt = x -> PATH * string(x) * "/"
NSUBJ = 29 # 7 in test runs
SIZEX = 91
SIZEY = 109
SIZEZ = 91
SHAPE = (SIZEX,SIZEY,SIZEZ)
SIZE = SIZEX * SIZEY * SIZEZ
LEN = 3620

3620

In [15]:
function get_nii(subj::Int, ind::Int)
    return niread(joinpath(PATH_Fmt(subj),FNAMES[subj][ind]),mmap= true)
end

function get_niis(subj::Int, rng::AbstractArray=1:LEN)
    fnames = readdir(PATH_Fmt(subj))
    return [niread(joinpath(PATH_Fmt(subj),fnames[ind]),mmap= true) for ind in rng]
end

@inline function rema!(shadow::Array, variable::Array,decay::Float64= 0.3)
    shadow -= (1 - decay) * (shadow - variable)
end

rema! (generic function with 2 methods)

In [54]:
@time D = distribute(disp = ans;get_niis(2,1:3620));

  5.564585 seconds (2.38 M allocations: 53.562 MB, 0.30% gc time)


In [55]:
m = niread(joinpath(PATH_Fmt(2),"mean.nii")).raw;

In [18]:
@time disp = mapfoldl(x->(x-m).*(x-m),+,D);

LoadError: ArgumentError: Module SharedArrays not found in current path.
Run `Pkg.add("SharedArrays")` to install the SharedArrays package.

In [20]:
@time corrs = zeros(SIZE, NSUBJ, NSUBJ); disps = zeros(corrs);

  1.315722 seconds (12.48 k allocations: 5.656 GB, 4.15% gc time)


In [15]:
@Logging.configure(level=DEBUG)

function macro_log_test()
    @debug("debug message")
    @info("info message")
    @warn("warning message")
    @err("error message")
    @critical("critical message")
end

macro_log_test()

22-май 17:04:35:DEBUG:root:debug message
22-май 17:04:35:INFO:root:info message
22-май 17:04:35:WARNING:root:warning message
22-май 17:04:35:ERROR:root:error message
22-май 17:04:35:CRITICAL:root:critical message
